#### Train DT network

In [2]:
import sys
sys.path.append("..")

from components.fcn import FCN
from components.datasets import ConesDataset

dataset_name = "dataset_no_cross_offline"

train_dataset = ConesDataset(f"F:/Data/ConeDatasets/{dataset_name}/train", train=True, convert_to_dt=True, pre_generated=True, augment_rotate=True)
train_dataloader = train_dataset.get_dataloader(shuffle=True,batch_size=32)

validation_dataset = ConesDataset(f"F:/Data/ConeDatasets/{dataset_name}/validation", train=False, convert_to_dt=True, pre_generated=True)
validation_dataloader = validation_dataset.get_dataloader(shuffle=False, batch_size=32)

#fcn = FCN(train_dataloader, validation_dataloader=validation_dataloader, lr=3e-4, depth=3, input_channels=1, initial_feature_maps=16, blocks_per_resolution_layer=2, use_MSE_loss=True)
#fcn = FCN(train_dataloader, validation_dataloader=validation_dataloader, lr=3e-4, depth=3, input_channels=1, initial_feature_maps=16, blocks_per_resolution_layer=4, use_MSE_loss=True)

fcn = FCN(train_dataloader, validation_dataloader=validation_dataloader, lr=3e-4, depth=3, input_channels=1, 
    initial_feature_maps=32, blocks_per_resolution_layer=2, use_MSE_loss=True, masked_MSE_loss=True)
#fcn = FCN(train_dataloader, validation_dataloader=validation_dataloader, lr=3e-4, depth=3, input_channels=1, initial_feature_maps=32, blocks_per_resolution_layer=3, use_MSE_loss=True)

#fcn = FCN(train_dataloader, validation_dataloader=validation_dataloader, lr=3e-4, depth=3, input_channels=1, initial_feature_maps=48, blocks_per_resolution_layer=2, use_MSE_loss=True)

fcn.train_network(epochs=100)

Average running TRAINING loss for epoch 1: 5.704558099138326
Average running VALIDATION loss for epoch 1: 5.878391209244728
Average running TRAINING loss for epoch 2: 2.24878742854143
Average running VALIDATION loss for epoch 2: 2.6218142323195934
Average running TRAINING loss for epoch 3: 0.7865463591598231
Average running VALIDATION loss for epoch 3: 2.1108170792460443
Average running TRAINING loss for epoch 4: 0.5782917836873696
Average running VALIDATION loss for epoch 4: 1.9916305169463158
Average running TRAINING loss for epoch 5: 0.587471532539047
Average running VALIDATION loss for epoch 5: 2.003329253941774
Average running TRAINING loss for epoch 6: 0.5537111788731197
Average running VALIDATION loss for epoch 6: 2.0273272044956685


KeyboardInterrupt: 

### Train DT network with k-fold cross validation

In [1]:
import sys
sys.path.append("..")

from components.fcn import FCN
from components.datasets import ConesDataset

# Config
k = 8 #5
epochs_per_fold = 300 #50 #25 #30 #50
dataset_name = "20230301_cross_eroded_equalized_64_cropped"
#"20230301_cross_eroded_equalized_cropped" #"20230301_cross_eroded_cropped"
#"cross_excluded_eroded_144_offline_added"
#"dataset_cross_no_BAK8095L_eroded_offline_low_plus" #"dataset_cross_no_BAK8095L_eroded_offline_512" #"dataset_cross_offline_eroded" # "dataset_cross_offline"

# Create dataset
dataset = ConesDataset(f"F:/Data/ConeDatasets/{dataset_name}/train", train=True, 
    convert_to_dt=True, #False,
    convert_to_gaussians=False, #True,
    pre_generated=True, 
    augment_rotate=True, 
    augment_blur=False, #True
    augment_elastic=False,
    adaptive_dilation=False,
    dilation = 1) 

# Create folds and save to disk (ONLY USE ONCE)
# dataset.get_k_fold_splits(k, "F:/Repositories/cone-detection-master-thesis/folds")

# # Train with a SINGLE fold
# fold_id = 0
# fold_path = f"F:/Repositories/cone-detection-master-thesis/folds/fold_{fold_id}.npz"
# train_dataloader, validation_dataloader = dataset.get_single_dataloader_cross_validation(fold_path, batch_size=32)
# FCN.train_network_single_cross_validation(fold_id, train_dataloader, validation_dataloader, max_epochs=epochs_per_fold, early_stopping=True)

# Train WITH all k folds
dataloaders = dataset.get_dataloaders_k_fold_cross_validation(k, batch_size=32) #48)
#FCN.train_networks_k_fold_cross_validation(dataloaders, max_epochs=epochs_per_fold, early_stopping=True)
FCN.train_networks_k_fold_cross_validation(dataloaders, max_epochs=epochs_per_fold, lr=3e-4, early_stopping=False, #True, 
                                           use_lr_scheduler=True) #,
#                                            use_distance_information=False, regularity_aware_training=False,
#                                            use_refiner=False) #True)

c:\Anaconda3\envs\cones\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  8%|▊         | 25/300 [2:37:29<28:52:20, 377.97s/it]


KeyboardInterrupt: 

### Train DT network without validation set

In [1]:
import sys
sys.path.append("..")

from components.fcn import FCN
from components.datasets import ConesDataset

# Config
dataset_name = "20230301_cross_eroded_cropped"

# Create dataset
dataset = ConesDataset(f"F:/Data/ConeDatasets/{dataset_name}/train", train=True, 
    convert_to_dt=True, #False,
    convert_to_gaussians=False, #True,
    pre_generated=True, 
    augment_rotate=True, 
    augment_blur=False, #True
    augment_elastic=False,
    adaptive_dilation=False,
    dilation = 1) 
dataloader = dataset.get_dataloader(batch_size=32)

model = FCN(dataloader, lr=1e-5, depth=3, input_channels=1, 
            initial_feature_maps=32, blocks_per_resolution_layer=2, 
            use_MSE_loss=True, masked_MSE_loss=True, use_class_weights=False,
            use_lr_scheduler=True) #, lr=3e-4)

model.load("../nets/DT-0.16-dilation-1-300-epochs.pth")

model.train_network(max_epochs=300, run_id="transfer-DT-0.16", verbose=False, early_stopping=False)

c:\Anaconda3\envs\cones\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
 47%|████▋     | 141/300 [14:41:13<16:33:43, 374.99s/it]


KeyboardInterrupt: 

In [1]:
### Transfer learning with low density

# import sys
# sys.path.append("..")
# import torch

# from components.fcn import FCN
# from components.datasets import ConesDataset

# max_epochs = 20
# early_stopping = True

# train_dataset = ConesDataset(f"F:/Data/ConeDatasets/confocal_npz", train=True, convert_to_dt=True, pre_generated=True, augment_rotate=True)
# train_dataloader = train_dataset.get_dataloader(shuffle=True,batch_size=32)

# validation_dataset = ConesDataset(f"F:/Data/ConeDatasets/dataset_cross_no_BAK8095L_eroded_offline/test", train=False, convert_to_dt=True, pre_generated=True)
# validation_dataloader = validation_dataset.get_dataloader(shuffle=False, batch_size=32)

# model = FCN(train_dataloader, validation_dataloader=validation_dataloader, lr=3e-4, depth=3, input_channels=1, 
#                 initial_feature_maps=32, blocks_per_resolution_layer=2, use_MSE_loss=True, masked_MSE_loss=True, use_class_weights=False)

# #model.load("../nets/5-fold-eroded-excluded-512-DT-0.23.pth")
            
# trained_epochs, min_training_loss, min_validation_loss = model.train_network(
#     max_epochs=max_epochs, 
#     run_id=f"transfer_learning", 
#     verbose=False, 
#     early_stopping=early_stopping)

# print(f"{trained_epochs} epochs, ({min_training_loss},{min_validation_loss})")
# del model
# torch.cuda.empty_cache()

c:\Anaconda3\envs\cones\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
 15%|█▌        | 3/20 [03:17<18:39, 65.86s/it]

4 epochs, (0.3335914104073136,0.4250504585603873)


In [1]:
# # Train Hamwood reference port in Python
# import sys
# sys.path.append("..")

# from components.hamwood import Hamwood
# from components.datasets import ConesDataset

# # Config
# k = 8
# epochs_per_fold = 25
# dataset_name = "20230301_cross_eroded_cropped"

# # Create dataset
# dataset = ConesDataset(f"F:/Data/ConeDatasets/{dataset_name}/train", train=True, 
#     convert_to_dt=False,
#     convert_to_gaussians=False,
#     pre_generated=True, 
#     augment_rotate=True, 
#     augment_blur=False,
#     augment_elastic=False,
#     adaptive_dilation=False,
#     dilation = 1) 

# # Train WITH all k folds
# dataloaders = dataset.get_dataloaders_k_fold_cross_validation(k, batch_size=32)
# Hamwood.train_networks_k_fold_cross_validation(dataloaders, max_epochs=epochs_per_fold, early_stopping=True)

c:\Anaconda3\envs\cones\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Anaconda3\envs\cones\lib\site-packages\torch\nn\modules\lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
 32%|███▏      | 8/25 [56:30<2:00:04, 423.77s/it]


KeyboardInterrupt: 

## Train FN classifier

In [1]:
import sys
sys.path.append("..")

from components.fn_classifier import FNDataset, FNClassifier

k = 8 #5
epochs_per_fold = 300 #50 #25 #30 #50
dataset_name = "FN_dataset"
#"20230301_cross_eroded_equalized_cropped" #"20230301_cross_eroded_cropped"
#"cross_excluded_eroded_144_offline_added"
#"dataset_cross_no_BAK8095L_eroded_offline_low_plus" #"dataset_cross_no_BAK8095L_eroded_offline_512" #"dataset_cross_offline_eroded" # "dataset_cross_offline"

# Create dataset
dataset = FNDataset(f"F:/Data/ConeDatasets/{dataset_name}/train", train=True)

# Train WITH all k folds
dataloaders = dataset.get_dataloaders_k_fold_cross_validation(k, batch_size=64)
#FCN.train_networks_k_fold_cross_validation(dataloaders, max_epochs=epochs_per_fold, early_stopping=True)
FNClassifier.train_networks_k_fold_cross_validation(dataloaders, max_epochs=epochs_per_fold, early_stopping=True)

c:\Anaconda3\envs\cones\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Anaconda3\envs\cones\lib\site-packages\torch\nn\modules\lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Average running TRAINING loss for epoch 1: 0.4336323504304064
Average running VALIDATION loss for epoch 1: 0.3319357826429255
Average running TRAINING loss for epoch 2: 0.3109303093675909
Average running VALIDATION loss for epoch 2: 0.28470051551566405
Average running TRAINING loss for epoch 3: 0.27993377837641487
Average running VALIDATION loss for epoch 3: 0.28295116038883433
Average running TRAINING loss for epoch 4: 0.26226563314939366
Average running VALIDATION loss for epoch 4: 0.2836194082218058
Average running TRAINING loss for epoch 5: 0.24813257793671098
Average running VALIDATION loss for epoch 5: 0.27203455655013814
Average running TRAINING loss for epoch 6: 0.23341428083849364
Average running VALIDATION loss for epoch 6: 0.2630419722374748
Average running TRAINING loss for epoch 7: 0.22970670949796151
Average running VALIDATION loss for epoch 7: 0.27344947001513314
Average running TRAINING loss for epoch 8: 0.22336993051757073
Average running VALIDATION loss for epoch 8: 0